In [407]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from typing import List
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
# nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from string import punctuation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")

In [4]:
df.sample(5)

,repository_name,commit_hash,commit_date,commit_author,commit_message,bugs
37,mlm,a8b55539,2020-03-08T17:19:19,Victor,Added .gitignore,3
304,sensor,f946e945,2020-04-01T12:27:44,Mallory,Added logrotate,1
293,sensor,4f2ef924,2020-04-02T13:52:38,Mallory,Added dns trap logs in filebeat config,4
177,conductor,8a99dcc4,2020-03-20T10:31:11,Dabe,More flexible removing of extension in compose...,6
120,conductor,1e6c10c7,2020-04-24T14:05:14,Trudy,keep_alive url changed,2


In [5]:
print(f"Размер датасета: {len(df)}")

Размер датасета: 319


Данных довольно мало....посмотрим на наличие пропусков и дубликатов

In [6]:
df.isnull().sum()

repository_name    0
commit_hash        0
commit_date        0
commit_author      0
commit_message     0
bugs               0
dtype: int64

In [7]:
df.duplicated().sum()

0

А чего от меня вообще хотят??? угу....тут задача регрессии...над предсказывать количество багов в зависимости от всех остальных параметров

Сначала посмотрим на признак commit_hash

In [8]:
len(df) == len(set(df.commit_hash.values))

True

хммммм значит все значения уникальны...данный признак удаляем, так как он не содержит никакой полезной информации для нас

In [9]:
df.drop(columns=["commit_hash"], inplace=True)

In [10]:
print(df.columns)
print(len(df))

Index(['repository_name', 'commit_date', 'commit_author', 'commit_message',
       'bugs'],
      dtype='object')
319


In [11]:
# посмотрим на количество уникальных элементов в ряде столбцов
for column_name in ["repository_name", "commit_author"]:
    print(f"Число уникальных элементов в столбце {column_name} - {len(df[column_name].unique())}")

Число уникальных элементов в столбце repository_name - 6
Число уникальных элементов в столбце commit_author - 10


Посмотрим суммарное количество ошибок для определенных авторов и репозиториев

In [12]:
print(df.groupby("repository_name")["bugs"].sum())
print()
print(df.groupby("commit_author")["bugs"].sum())

repository_name
agent         41
conductor    343
dockers       48
mlm          206
sensor       233
standard      85
Name: bugs, dtype: int64

commit_author
Alice        1
Bob          5
Carol        6
Dabe       101
Eve         32
Mallory    213
Peggy        7
Trudy       19
Victor     464
Wendy      108
Name: bugs, dtype: int64


Тут напрашивается OHE, но пока оставим как есть, ибо данных оч мало, и прост закодируем числами для применения деревянной модели

Но сначала посмотрим на дату и описание коммита

In [13]:
date_from_df = pd.to_datetime(df.commit_date)

In [14]:
date_from_df[0]

Timestamp('2020-06-12 14:26:12')

In [15]:
print(date_from_df.apply(lambda x: x.year).value_counts())
print()
print(date_from_df.apply(lambda x: x.month).value_counts())
print()
print(date_from_df.apply(lambda x: x.day).value_counts())


2020    319
Name: commit_date, dtype: int64

4    122
5     83
3     65
6     49
Name: commit_date, dtype: int64

1     32
10    24
24    22
8     17
13    16
15    15
14    15
23    15
30    14
9     14
22    13
6     13
20    13
2     11
4      9
21     9
29     9
5      9
3      8
16     7
7      7
12     6
11     6
28     6
17     4
19     2
27     1
18     1
25     1
Name: commit_date, dtype: int64


In [16]:
temp_date = date_from_df[0]

In [17]:
temp_date.day_of_week

4

In [18]:
temp_date.hour

14

Чего мне делать с временем??? мне над разбить день на опредленные части также учитывать месяц год учитывать не над

так как он одинаковый везде хммммм день недели мне нужен?

Теперь посмотрим на сообщение в коммите

In [24]:
print(df.commit_message.values[:100])

['объяснения предсказаний с помощью lime' 'Поставил lime вместо eli5'
 'Скрипт для создния пакетов обновления' 'Сортировка весов по убыванию'
 'не надо конвертировать массив до операции по смене знака'
 'Добавлено больше логирования и фикс до частичной работоспособности'
 'Отладка, чтоб оно хотябы запускалось'
 'Добавил скрипт для запуска контейнера' 'label не нужен'
 'Установка всех зависимостей теперь в докер файле' 'Использование sha256'
 'Обновление схем'
 'Не сохранять неизвестные поля, на них нельзя обучаться и поэтому нет смысла их хранить'
 'hdf5 не умеет хранить матрицы различных типов, сделал группу с хешами, а в...'
 'Скрипт для создания датасета из csv файла'
 'тип и всякие статичные данные пусть лежат в датасете'
 'eli5 предсказание для бинарных классификаторов' 'Функции проверки меток'
 'Апи для моделей и датасетов'
 'Пересмотрели архитектуру, теперь все запросы через http'
 'Перешли с npz на hdf5' 'добавил aiohttp'
 'Диаграмы процессов и архитектуры' 'Оптимизировал докер

Попробуем самую простую токенизацию по пробельным символам

In [376]:
en_stops = set(stopwords.words('english'))
ru_stops = set(stopwords.words('russian'))
stops = en_stops | ru_stops

def tokenize(in_str: str) -> List[str]:
    temp_tokenize = [item for item in wordpunct_tokenize(in_str) if item not in punctuation]
    return [item for item in temp_tokenize if item not in stops]
    

In [391]:
vectorizer = TfidfVectorizer(tokenizer=tokenize)
vec_texts = vectorizer.fit_transform(df.commit_message).toarray()
svd = TruncatedSVD(n_components=15, random_state=42)
pca = PCA(n_components=50, random_state=42)
# svd_vec = svd.fit_transform(vec_texts)
svd_vec = pca.fit_transform(vec_texts)

In [392]:
svd_vec.shape

(319, 50)

In [393]:
df

,repository_name,commit_date,commit_author,commit_message,bugs,month,week,day,dayOfWeek,hour,dayOfWeek_,dayPart,dayPartLogic
0,mlm,2020-06-12 14:26:12,Victor,объяснения предсказаний с помощью lime,4,6,24,12,4,14,1,2,2.0
1,mlm,2020-06-12 11:53:10,Victor,Поставил lime вместо eli5,3,6,24,12,4,11,1,2,2.0
2,mlm,2020-06-11 15:19:03,Victor,Скрипт для создния пакетов обновления,4,6,24,11,3,15,1,2,2.0
3,mlm,2020-06-11 10:42:29,Victor,Сортировка весов по убыванию,3,6,24,11,3,10,1,2,2.0
4,mlm,2020-06-11 09:25:19,Victor,не надо конвертировать массив до операции по с...,7,6,24,11,3,9,1,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,sensor,2020-03-23 12:54:35,Dabe,setting hostname,1,3,13,23,0,12,1,2,2.0
315,sensor,2020-03-23 12:30:52,Dabe,heralding default dockerfile,2,3,13,23,0,12,1,2,2.0
316,sensor,2020-03-23 11:27:40,Dabe,unnesserity /data path permissions setting,3,3,13,23,0,11,1,2,2.0
317,sensor,2020-03-23 11:20:50,Dabe,added trapconductor,1,3,13,23,0,11,1,2,2.0


Что я могу сделать с временем...

In [394]:
df.commit_date = pd.to_datetime(df.commit_date)

In [395]:
df["month"] = df.commit_date.apply(lambda x: x.month)
df["week"] = df.commit_date.apply(lambda x: x.week)
df["day"] = df.commit_date.apply(lambda x: x.day)
df["dayOfWeek"] = df.commit_date.apply(lambda x: x.weekday())
df["hour"] = df.commit_date.apply(lambda x: x.hour)

In [396]:
df.hour.value_counts()

13    44
16    39
11    37
12    36
15    31
14    30
10    30
17    24
9     19
18     8
8      5
21     4
20     4
19     3
23     2
6      1
7      1
22     1
Name: hour, dtype: int64

In [397]:
def get_part_day_work(x: int) -> int:
    if x < 9 or x > 17:
        return 1
    else:
        return 2


def get_part_day_logic(x: int) -> int:
    if x > 3 and x < 10:
        return 1
    if x >= 10 and x < 18:
        return 2
    if x >= 18 and x < 23:
        return 3
    if x >= 23 and x <= 3:
        return 4

In [398]:
import razdel
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

In [399]:
df["dayOfWeek_"] = df.dayOfWeek.apply(lambda x: 1 if x in [0, 1, 2, 3, 4] else 2)
df["dayPart"] = df.hour.apply(lambda x: get_part_day_work(x))
df["dayPartLogic"] = df.hour.apply(lambda x: get_part_day_logic(x))


temp = model.encode(df.commit_message.values)

pca_emb = PCA(n_components=10)
low_emb = pd.DataFrame(pca_emb.fit_transform(temp))

In [400]:
# X_train, X_test, y_train, y_test = train_test_split(vec_texts.toarray(), df.bugs.values)
# new_df = pd.concat([pd.get_dummies(df.repository_name), pd.get_dummies(df.commit_author), pd.DataFrame(svd_vec)], axis=1)
# new_df = pd.concat([pd.get_dummies(df.repository_name), pd.get_dummies(df.commit_author), pd.get_dummies(df.dayOfWeek_),
# pd.get_dummies(df.dayPart), pd.DataFrame(svd_vec)], axis=1)

new_df = pd.concat([pd.get_dummies(df.repository_name), pd.get_dummies(df.commit_author), pd.get_dummies(df.dayOfWeek_),
pd.get_dummies(df.dayPart), low_emb], axis=1)

X_train, X_test, y_train, y_test = train_test_split(new_df, df.bugs.values, random_state=42)
# pd.get_dummies(df.commit_author)



In [401]:
X_train.shape

(239, 30)

In [402]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

c:\ProgramData\Anaconda3\envs\diplomaProject\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\diplomaProject\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\diplomaProject\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [408]:
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [409]:
mean_squared_error(y_test, gb.predict(X_test))

1.0791168366919504

In [355]:
lr.predict(X_test)

array([ 2.52300454e+00,  1.49227315e+00,  6.19487954e+00,  3.94487954e+00,
        5.17144204e+00,  3.65581704e+00,  2.42144204e+00,  2.42144204e+00,
        3.15581704e+00,  2.86727315e+00,  1.85112954e+00,  2.52300454e+00,
        2.42144204e+00,  2.42144204e+00,  2.60894204e+00,  2.99175454e+00,
        2.42144204e+00,  3.14019204e+00,  1.12456704e+00,  2.35894204e+00,
        2.81206704e+00,  2.42144204e+00,  2.42144204e+00,  3.65581704e+00,
        2.02300454e+00,  2.58550454e+00,  2.11727315e+00,  5.17144204e+00,
        3.65581704e+00,  2.42144204e+00,  3.14019204e+00,  1.89800454e+00,
        2.42144204e+00,  2.35894204e+00,  3.78081704e+00,  2.52300454e+00,
        2.42144204e+00,  2.42144204e+00,  3.14019204e+00,  3.44487954e+00,
        5.17144204e+00,  5.06987954e+00,  1.85112954e+00,  1.85112954e+00,
       -1.61661250e+14,  5.17144204e+00,  3.14019204e+00,  2.35894204e+00,
        2.35894204e+00,  2.80477315e+00,  3.15581704e+00,  2.83550454e+00,
        2.42144204e+00, -

Хочу посмотреть, как меняется количество ошибок для каждого репозитория по времени